In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical

from sklearn.ensemble import RandomForestClassifier

### Abrindo o dataset de treino

In [2]:
df_application = pd.read_csv('application_train.csv')

In [3]:
df_application

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,456162,0,Cash loans,F,N,N,0,112500.0,700830.0,22738.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,134978,0,Cash loans,F,N,N,0,90000.0,375322.5,14422.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
2,318952,0,Cash loans,M,Y,N,0,180000.0,544491.0,16047.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,3.0
3,361264,0,Cash loans,F,N,Y,0,270000.0,814041.0,28971.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
4,260639,0,Cash loans,F,N,Y,0,144000.0,675000.0,21906.0,...,0,0,0,0,0.0,0.0,0.0,10.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246003,242114,0,Cash loans,F,N,Y,1,270000.0,1172470.5,34411.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,8.0
246004,452374,0,Cash loans,F,N,Y,0,180000.0,654498.0,27859.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
246005,276545,1,Revolving loans,M,N,N,1,112500.0,270000.0,13500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
246006,236776,1,Cash loans,M,Y,N,3,202500.0,204858.0,17653.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


### Tratando algumas variáveis do modelo

In [4]:
# Pegando os valores da feature CODE GENDER diferentes de XNA
df_application = df_application[df_application['CODE_GENDER'] != 'XNA']

In [5]:
# Removendo outlier da variável AMT_INCOME_TOTAL
df_application = df_application[df_application['AMT_INCOME_TOTAL'] < 20000000]

In [6]:
# Tratando a feature DAYS_EMPLOYED
df_application['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

In [7]:
# Tratando a feature DAYS_LAST_PHONE_CHANGE
df_application['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

In [8]:
# Categorizando a idade
def categoriza_idade(dia):
        idade_anos = -dia / 365
        if idade_anos < 27: return 1
        elif idade_anos < 40: return 2
        elif idade_anos < 50: return 3
        elif idade_anos < 65: return 4
        elif idade_anos < 99: return 5
        else: return 0
    
df_application['AGE_RANGE'] = df_application['DAYS_BIRTH'].apply(lambda x: categoriza_idade(x))

In [9]:
# Categorizando a feture acompanhantes
def trata_acompanhante_(valor):
    if valor in ['Group of people', 'Other_B']:
        return 1
    elif valor in ['Unaccompanied', 'Other_A']:
        return 2
    elif valor in ['Family', 'Spouse, partner', 'Children']:
        return 3
    else:
        return np.nan
    
df_application['NAME_TYPE_SUITE'] = df_application['NAME_TYPE_SUITE'].apply(lambda x: trata_acompanhante_(x))

In [10]:
# Categorizando a feture tipo de renda
def trata_acompanhante_(valor):
    if valor in ['Working']:
        return 1
    elif valor in ['Commercial associate']:
        return 2
    else:
        return 3
    
df_application['NAME_INCOME_TYPE'] = df_application['NAME_INCOME_TYPE'].apply(lambda x: trata_acompanhante_(x))

In [11]:
def trata_educacao_(valor):
    if valor in ['Lower secondary']:
        return 1
    elif valor in ['Secondary / secondary special', 'Incomplete higher']:
        return 2
    elif valor in ['Higher education']:
        return 3
    elif valor in ['Academic degree']:
        return 4
    else:
        return -1

In [12]:
def trata_educacao(tabela):
    tabela = tabela.copy()
    for i in tabela.columns:
        tabela[i] = tabela[i].apply(trata_educacao_)
    return tabela

## Analisando e separando as variaveis importantes do dataset bureau

In [13]:
df_bureau = pd.read_csv('bureau.csv')

In [14]:
# criando a coluna da duração do crédito = dias antes do pedido -  duração restante
df_bureau['CREDIT_DURATION_DAYS'] = -df_bureau['DAYS_CREDIT'] + df_bureau['DAYS_CREDIT_ENDDATE']

In [15]:
# Categorizando a df_junto, se não deveu é 0 senão é 1
df_bureau['CREDIT_DAY_OVERDUE'] = df_bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 0 else 0)

In [16]:
# Dropando algumas features
df_reduzido = df_bureau[['SK_ID_CURR', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_SUM', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'CREDIT_DURATION_DAYS']]

## Juntando os datasets bureau + application train

In [17]:
df_junto = pd.merge(left=df_application, right=df_reduzido, how = 'inner', on = 'SK_ID_CURR')

### Acrescentando as variaveis criadas antes

In [18]:
# Criando uma variavel que representa o quanto o crédito fornecido representa da receita total do devedor
df_junto['DEBT_TO_INCOME'] = df_junto['AMT_CREDIT']/df_junto['AMT_INCOME_TOTAL']

In [19]:
# Criando uma variável que representa o quanto sobra da receita do cliente após pagamento da anuidade
df_junto['INTEREST_COVER'] = df_junto['AMT_INCOME_TOTAL'] / df_junto['AMT_ANNUITY']

In [20]:
# Criando uma variável que representa o quanto do crédito representa preço do ativo
df_junto['DOWN_PAYMENT'] = df_junto['AMT_CREDIT'] / df_junto['AMT_GOODS_PRICE'] -1

In [21]:
# Criando uma variável que representa a porcentagem total da renda por cada membro
df_junto['TOTAL_INCOME_BY_MENBERS'] = df_junto['AMT_INCOME_TOTAL'] / df_junto['CNT_FAM_MEMBERS']

### Dropando algumas features

In [22]:
df_junto = df_junto.drop(['AGE_RANGE','CODE_GENDER','ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG',
                          'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE',
                          'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI',
                          'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
                          'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13',
                          'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
                          'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
                          'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
                          'AMT_REQ_CREDIT_BUREAU_YEAR'], axis=1)

In [23]:
df_junto

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_SUM,CREDIT_TYPE,DAYS_CREDIT_UPDATE,CREDIT_DURATION_DAYS,DEBT_TO_INCOME,INTEREST_COVER,DOWN_PAYMENT,TOTAL_INCOME_BY_MENBERS
0,456162,0,Cash loans,N,N,0,112500.0,700830.0,22738.5,585000.0,...,378.0,-201.0,54634.500,Consumer credit,-196,730.0,6.229600,4.947556,0.1980,112500.0
1,456162,0,Cash loans,N,N,0,112500.0,700830.0,22738.5,585000.0,...,-176.0,-174.0,225000.000,Consumer credit,-174,348.0,6.229600,4.947556,0.1980,112500.0
2,456162,0,Cash loans,N,N,0,112500.0,700830.0,22738.5,585000.0,...,1543.0,NaN,180000.000,Credit card,-6,1794.0,6.229600,4.947556,0.1980,112500.0
3,456162,0,Cash loans,N,N,0,112500.0,700830.0,22738.5,585000.0,...,NaN,NaN,292500.000,Credit card,-21,NaN,6.229600,4.947556,0.1980,112500.0
4,456162,0,Cash loans,N,N,0,112500.0,700830.0,22738.5,585000.0,...,116.0,-188.0,42075.000,Consumer credit,-186,365.0,6.229600,4.947556,0.1980,112500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173778,452374,0,Cash loans,N,Y,0,180000.0,654498.0,27859.5,585000.0,...,-1367.0,-1367.0,26190.000,Consumer credit,-1363,275.0,3.636100,6.460992,0.1188,90000.0
1173779,452374,0,Cash loans,N,Y,0,180000.0,654498.0,27859.5,585000.0,...,109.0,NaN,118080.000,Consumer credit,-12,365.0,3.636100,6.460992,0.1188,90000.0
1173780,452374,0,Cash loans,N,Y,0,180000.0,654498.0,27859.5,585000.0,...,282.0,NaN,206480.385,Consumer credit,-24,366.0,3.636100,6.460992,0.1188,90000.0
1173781,236776,1,Cash loans,Y,N,3,202500.0,204858.0,17653.5,171000.0,...,-2341.0,-2334.0,33057.000,Consumer credit,-513,184.0,1.011644,11.470813,0.1980,40500.0


## Analisando e separando as variaveis importantes do dataset pos_cash

In [24]:
df_CASH = pd.read_csv('POS_CASH_balance.csv')

In [25]:
# Categorizando, se deve = 1 senão = 0
df_CASH['SK_DPD'] = df_CASH['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)

In [26]:
# Pegando somente dados até seis meses
df_CASH = df_CASH[df_CASH['MONTHS_BALANCE'] >= -6]

In [27]:
# Pegando somente três features
df_CASH = df_CASH[['SK_ID_CURR','SK_DPD', 'CNT_INSTALMENT_FUTURE']]

In [28]:
df_CASH

,SK_ID_CURR,SK_DPD,CNT_INSTALMENT_FUTURE
701,146563,0,12.0
702,444611,0,8.0
703,270883,0,10.0
704,450849,0,14.0
705,203479,0,14.0
...,...,...,...
10001012,307071,0,0.0
10001022,390187,0,12.0
10001023,204579,0,12.0
10001024,182132,0,0.0


## Analisando e separando as variaveis importantes do dataset credit_card_balance

In [29]:
df_credit = pd.read_csv('credit_card_balance.csv')

In [30]:
# Renomeando coluna
df_credit.rename(columns={'SK_DPD': 'SK_DPD_CREDIT'}, inplace=True)

In [31]:
df_credit = df_credit.drop(['AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 
                'AMT_PAYMENT_CURRENT', 'AMT_RECIVABLE', 'SK_DPD_DEF', 'NAME_CONTRACT_STATUS',
                'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 
                'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM'], axis=1)

In [32]:
df_credit

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_TOTAL_RECEIVABLE,SK_DPD_CREDIT
0,2562384,378907,-6,56.970,135000,877.5,1700.325,1800.000,0.000,0.000,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.000,2250.000,60175.080,64875.555,0
2,1740877,371185,-7,31815.225,450000,0.0,2250.000,2250.000,26926.425,31460.085,0
3,1389973,337855,-4,236572.110,225000,2250.0,11795.760,11925.000,224949.285,233048.970,0
4,1891521,126868,-1,453919.455,450000,11547.0,22924.890,27000.000,443044.395,453919.455,0
...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,0.0,0.000,0.000,0.000,0.000,0
3840308,1714892,347207,-9,0.000,45000,0.0,0.000,0.000,0.000,0.000,0
3840309,1302323,215757,-9,275784.975,585000,270000.0,2250.000,356994.675,269356.140,273093.975,0
3840310,1624872,430337,-10,0.000,450000,0.0,0.000,0.000,0.000,0.000,0


## Analisando e separando as variaveis importantes do dataset previous_application

In [33]:
df_previous = pd.read_csv('previous_application.csv')

In [34]:
# Arrumando as variaveis de dias
df_previous['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
df_previous['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
df_previous['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
df_previous['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
df_previous['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)

In [35]:
# Binarizando o valor de resposta
df_previous['FLAG_LAST_APPL_PER_CONTRACT'] = df_previous['FLAG_LAST_APPL_PER_CONTRACT'].map({'Y': 1, 'N': 0})

In [36]:
# Criando a variavel da diferença do valor que o cliente pediu para o valor que ele recebeu
df_previous['DIFFERENCE_RECEIVED'] = df_previous['AMT_APPLICATION'] - df_previous['AMT_CREDIT']

In [37]:
# Categorizando a coluna NAME_CASH_LOAN_PORPUSE
def trata_NAME_CASH(valor):
    if valor in ['Refusal to name the goal']:
        return 1
    elif valor in ['Car repairs', 'Money for a third person', 'Gasification / water supply', 'Hobby']:
        return 2
    elif valor in ['Payments on other loans', 'Urgent needs']:
        return 3
    elif valor in ['Building a house or an annex', 'Buying a home', 'Repairs', 'Buying a used car', 'Other',
                   'Purchase of electronic equipment', 'Wedding / gift / holiday', 'Buying a holiday home / land', 'Medicine']:
        return 4
    elif valor in ['Business development', 'Education', 'Everyday expenses', 'Furniture', 'Journey']:
        return 5
    elif valor in ['XAP', 'XNA', 'Buying a new car', 'Buying a garage']:
        return 6
    else:
        return valor
    
df_previous['NAME_CASH_LOAN_PURPOSE'] = df_previous['NAME_CASH_LOAN_PURPOSE'].apply(lambda x: trata_NAME_CASH(x))

In [38]:
df_previous = df_previous[['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_STATUS', 'CODE_REJECT_REASON', 'NAME_PORTFOLIO', 'NAME_YIELD_GROUP',
             'NAME_CASH_LOAN_PURPOSE', 'HOUR_APPR_PROCESS_START', 'RATE_DOWN_PAYMENT', 'AMT_DOWN_PAYMENT', 'AMT_ANNUITY',
             'DIFFERENCE_RECEIVED', 'FLAG_LAST_APPL_PER_CONTRACT']]

In [39]:
df_previous

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_STATUS,CODE_REJECT_REASON,NAME_PORTFOLIO,NAME_YIELD_GROUP,NAME_CASH_LOAN_PURPOSE,HOUR_APPR_PROCESS_START,RATE_DOWN_PAYMENT,AMT_DOWN_PAYMENT,AMT_ANNUITY,DIFFERENCE_RECEIVED,FLAG_LAST_APPL_PER_CONTRACT
0,2030495,271877,Approved,XAP,POS,middle,6,15,0.000000,0.0,1730.430,0.0,1
1,2802425,108129,Approved,XAP,Cash,low_action,6,11,NaN,NaN,25188.615,-72171.0,1
2,2523466,122040,Approved,XAP,Cash,high,6,11,NaN,NaN,15060.735,-23944.5,1
3,2819243,176158,Approved,XAP,Cash,middle,6,7,NaN,NaN,47041.335,-20790.0,1
4,1784265,202054,Refused,HC,Cash,high,4,9,NaN,NaN,31924.395,-66555.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Approved,XAP,POS,low_normal,6,12,0.000000,0.0,14704.290,-44104.5,1
1670210,2357031,334635,Approved,XAP,POS,middle,6,15,0.340554,29250.0,6622.020,23458.5,1
1670211,2659632,249544,Approved,XAP,POS,low_normal,6,12,0.101401,10525.5,11520.855,2713.5,1
1670212,2785582,400317,Approved,XAP,Cash,low_normal,6,9,NaN,NaN,18821.520,-11880.0,1


## Juntando alguns datasets 

In [40]:
# Juntando previous com o cartão de credito
df_junto_previous = pd.merge(left=df_previous, right=df_credit, how = 'inner', on = ['SK_ID_PREV', 'SK_ID_CURR'])

In [41]:
# Não agrupei este dataset pois aumenta muito as linhas do dataset
#df_junto_previous = pd.merge(left=df_junto_previous, right=df_CASH, how = 'inner', on = 'SK_ID_CURR')

In [42]:
# Pegando as informações dos ultimos seis meses
df_junto_previous = df_junto_previous[df_junto_previous['MONTHS_BALANCE']>= -6]

In [43]:
df_junto_previous

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_STATUS,CODE_REJECT_REASON,NAME_PORTFOLIO,NAME_YIELD_GROUP,NAME_CASH_LOAN_PURPOSE,HOUR_APPR_PROCESS_START,RATE_DOWN_PAYMENT,AMT_DOWN_PAYMENT,...,FLAG_LAST_APPL_PER_CONTRACT,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_TOTAL_RECEIVABLE,SK_DPD_CREDIT
0,1285768,142748,Approved,XAP,Cards,XNA,6,13,NaN,NaN,...,1,-1,181151.820,180000,0.000,9429.570,11250.000,177341.805,181151.820,0
1,1285768,142748,Approved,XAP,Cards,XNA,6,13,NaN,NaN,...,1,-4,184116.150,180000,2250.000,9380.745,11250.000,178368.570,182299.365,0
2,1285768,142748,Approved,XAP,Cards,XNA,6,13,NaN,NaN,...,1,-2,185366.655,180000,4500.000,9329.985,11250.000,179849.385,183796.155,0
3,1285768,142748,Approved,XAP,Cards,XNA,6,13,NaN,NaN,...,1,-5,184411.125,180000,900.000,9585.270,11250.000,177897.555,181852.875,0
4,1285768,142748,Approved,XAP,Cards,XNA,6,13,NaN,NaN,...,1,-6,186012.585,180000,1800.000,9661.680,13500.000,178794.495,182603.295,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2757488,2101747,339383,Approved,XAP,Cards,XNA,6,18,NaN,NaN,...,1,-4,137968.830,202500,6750.000,5480.910,9.180,132643.080,135277.830,0
2757489,2101747,339383,Approved,XAP,Cards,XNA,6,18,NaN,NaN,...,1,-3,147663.315,202500,11590.425,6965.820,128.565,142817.085,146317.815,0
2757491,2101747,339383,Approved,XAP,Cards,XNA,6,18,NaN,NaN,...,1,-5,131831.865,202500,29218.500,4229.055,144.090,127210.185,130486.365,0
2757494,2101747,339383,Approved,XAP,Cards,XNA,6,18,NaN,NaN,...,1,-2,153398.745,202500,6750.000,7521.435,103.320,148093.920,150707.745,0


### Juntando df_junto + outros

In [44]:
df_junto = pd.merge(left=df_junto, right=df_junto_previous, how = 'inner', on = 'SK_ID_CURR')

In [45]:
df_junto

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY_x,AMT_GOODS_PRICE,...,FLAG_LAST_APPL_PER_CONTRACT,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_TOTAL_RECEIVABLE,SK_DPD_CREDIT
0,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-6,0.000,405000,0.0,0.000,18.540,0.00,0.000,0
1,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-4,0.000,405000,0.0,0.000,18.540,0.00,0.000,0
2,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-5,0.000,405000,0.0,0.000,18.540,0.00,0.000,0
3,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-3,0.000,405000,0.0,0.000,18.540,0.00,0.000,0
4,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-2,9685.440,405000,9630.0,0.000,0.000,9611.46,9611.460,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1706423,454197,0,Cash loans,N,Y,2,81000.0,547344.0,23139.0,472500.0,...,1,-6,260429.310,427500,49500.0,8872.920,1975.005,252000.00,257738.310,0
1706424,454197,0,Cash loans,N,Y,2,81000.0,547344.0,23139.0,472500.0,...,1,-2,464168.205,427500,4500.0,22119.075,22050.000,424206.45,459556.290,28
1706425,454197,0,Cash loans,N,Y,2,81000.0,547344.0,23139.0,472500.0,...,1,-3,461468.835,427500,45000.0,19346.940,20250.000,423855.00,454237.965,0
1706426,454197,0,Cash loans,N,Y,2,81000.0,547344.0,23139.0,472500.0,...,1,-5,350723.745,427500,72000.0,11524.995,0.000,324000.00,343124.145,0


In [46]:
# Pegando as informações dos ultimos seis meses
df_junto = df_junto[df_junto['MONTHS_BALANCE']>= -6]

In [47]:
df_junto

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY_x,AMT_GOODS_PRICE,...,FLAG_LAST_APPL_PER_CONTRACT,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_TOTAL_RECEIVABLE,SK_DPD_CREDIT
0,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-6,0.000,405000,0.0,0.000,18.540,0.00,0.000,0
1,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-4,0.000,405000,0.0,0.000,18.540,0.00,0.000,0
2,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-5,0.000,405000,0.0,0.000,18.540,0.00,0.000,0
3,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-3,0.000,405000,0.0,0.000,18.540,0.00,0.000,0
4,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-2,9685.440,405000,9630.0,0.000,0.000,9611.46,9611.460,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1706423,454197,0,Cash loans,N,Y,2,81000.0,547344.0,23139.0,472500.0,...,1,-6,260429.310,427500,49500.0,8872.920,1975.005,252000.00,257738.310,0
1706424,454197,0,Cash loans,N,Y,2,81000.0,547344.0,23139.0,472500.0,...,1,-2,464168.205,427500,4500.0,22119.075,22050.000,424206.45,459556.290,28
1706425,454197,0,Cash loans,N,Y,2,81000.0,547344.0,23139.0,472500.0,...,1,-3,461468.835,427500,45000.0,19346.940,20250.000,423855.00,454237.965,0
1706426,454197,0,Cash loans,N,Y,2,81000.0,547344.0,23139.0,472500.0,...,1,-5,350723.745,427500,72000.0,11524.995,0.000,324000.00,343124.145,0


In [48]:
df_junto.SK_ID_CURR.value_counts()

120860    464
347816    456
177014    336
218175    330
161692    324
         ... 
321007      1
206629      1
176678      1
160748      1
153599      1
Name: SK_ID_CURR, Length: 53797, dtype: int64

In [49]:
df_junto_drop = df_junto.copy()

In [50]:
# Dropando os valores repetidos
df_junto_drop = df_junto_drop.drop_duplicates(subset=['SK_ID_CURR', 'MONTHS_BALANCE'], keep='last')

In [51]:
df_junto_drop

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY_x,AMT_GOODS_PRICE,...,FLAG_LAST_APPL_PER_CONTRACT,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_TOTAL_RECEIVABLE,SK_DPD_CREDIT
40,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-6,0.000,405000,0.0,0.000,18.540,0.00,0.000,0
41,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-4,0.000,405000,0.0,0.000,18.540,0.00,0.000,0
42,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-5,0.000,405000,0.0,0.000,18.540,0.00,0.000,0
43,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-3,0.000,405000,0.0,0.000,18.540,0.00,0.000,0
44,134978,0,Cash loans,N,N,0,90000.0,375322.5,14422.5,324000.0,...,1,-2,9685.440,405000,9630.0,0.000,0.000,9611.46,9611.460,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1706423,454197,0,Cash loans,N,Y,2,81000.0,547344.0,23139.0,472500.0,...,1,-6,260429.310,427500,49500.0,8872.920,1975.005,252000.00,257738.310,0
1706424,454197,0,Cash loans,N,Y,2,81000.0,547344.0,23139.0,472500.0,...,1,-2,464168.205,427500,4500.0,22119.075,22050.000,424206.45,459556.290,28
1706425,454197,0,Cash loans,N,Y,2,81000.0,547344.0,23139.0,472500.0,...,1,-3,461468.835,427500,45000.0,19346.940,20250.000,423855.00,454237.965,0
1706426,454197,0,Cash loans,N,Y,2,81000.0,547344.0,23139.0,472500.0,...,1,-5,350723.745,427500,72000.0,11524.995,0.000,324000.00,343124.145,0


In [52]:
df_junto_drop.SK_ID_CURR.value_counts()

106548    6
415833    6
123054    6
125103    6
178373    6
         ..
206629    1
302040    1
120752    1
181267    1
231943    1
Name: SK_ID_CURR, Length: 53797, dtype: int64

### Rodando o modelo com o dataset junto

In [53]:
var_cat = ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY',  'NAME_FAMILY_STATUS', 
           'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
           'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE', 'CREDIT_TYPE', 'NAME_CONTRACT_STATUS',
           'CODE_REJECT_REASON', 'NAME_PORTFOLIO', 'NAME_YIELD_GROUP']
var_edu = ['NAME_EDUCATION_TYPE']
var_num = [i for i in df_junto_drop.columns[3:] if df_junto_drop[i].dtype != 'object']

In [54]:
df_treino, df_teste = train_test_split(df_junto_drop, test_size = 0.2, random_state = 1)
df_treino, df_teste = df_treino.copy(), df_teste.copy()

x_treino = df_treino.loc[:, var_edu + var_cat + var_num]
x_teste = df_teste.loc[:, var_edu + var_cat + var_num]

y_treino = df_treino.loc[:, 'TARGET']
y_teste = df_teste.loc[:, 'TARGET']

In [55]:
pipe_edu = Pipeline(steps = [
    ('educacao', FunctionTransformer(trata_educacao))
])

# para não ser necessário fazer o impute dos missing, transformo em string e trato como uma nova categoria
pipe_cat = Pipeline(steps = [
    ('para_string', FunctionTransformer(lambda x: x.astype(str))), 
    ('dummies', OneHotEncoder(drop = 'if_binary', sparse = False)),
])

pipe_num = Pipeline(steps = [
    ('redimensionamento', StandardScaler()),
    ('imputer', SimpleImputer(strategy = 'median', add_indicator = 'True'))
])

trata_variaveis = ColumnTransformer(transformers = [
    ('pipe_edu', pipe_edu, var_edu),
    ('pipe_cat', pipe_cat, var_cat),
    ('pipe_num', pipe_num, var_num)
])

pipe = Pipeline(steps = [
    ('pre_processamento', trata_variaveis),
    ('feature_selection', SelectFromModel(LogisticRegression(penalty = 'l1', 
                                                             solver = 'liblinear', 
                                                             C = 0.01))),
    
    ('modelo', RandomForestClassifier(
        criterion='entropy',
        n_estimators = 100,
        max_samples = 0.9,
        max_features = 0.6,
        max_depth = 9,
        n_jobs = 10,
        random_state = 1))
])

In [56]:
%%time
pipe.fit(x_treino, y_treino)

Wall time: 1min 25s


Pipeline(steps=[('pre_processamento',
                 ColumnTransformer(transformers=[('pipe_edu',
                                                  Pipeline(steps=[('educacao',
                                                                   FunctionTransformer(func=<function trata_educacao at 0x000001A90D9D80D0>))]),
                                                  ['NAME_EDUCATION_TYPE']),
                                                 ('pipe_cat',
                                                  Pipeline(steps=[('para_string',
                                                                   FunctionTransformer(func=<function <lambda> at 0x000001A90CEE2670>)),
                                                                  ('dummies',
                                                                   OneHotEncoder(drop='if...
                                                   'LIVE_REGION_NOT_WORK_REGION',
                                                   'REG_CITY_NOT_L

In [57]:
y_treino_pred = pipe.predict_proba(x_treino)[:, 1]
y_teste_pred = pipe.predict_proba(x_teste)[:, 1]

In [58]:
roc_auc_score(y_treino, y_treino_pred)

0.8750999862545241

In [59]:
roc_auc_score(y_teste, y_teste_pred)

0.8539597178178174

In [60]:
df_amostra = df_junto.sample(n=7)

In [61]:
df_amostra

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY_x,AMT_GOODS_PRICE,...,FLAG_LAST_APPL_PER_CONTRACT,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_TOTAL_RECEIVABLE,SK_DPD_CREDIT
73732,116081,0,Cash loans,Y,Y,2,180000.0,889515.0,26005.5,742500.0,...,1,-1,0.00,45000,0.0,0.000,0.000,0.000,0.00,0
890254,313981,0,Cash loans,N,N,0,94500.0,360000.0,17509.5,360000.0,...,1,-6,40037.76,90000,0.0,4500.000,54000.000,37581.525,40037.76,0
921161,429701,0,Cash loans,Y,N,0,112500.0,540000.0,30280.5,540000.0,...,1,-5,0.00,675000,0.0,0.000,0.000,0.000,0.00,0
491063,356487,0,Cash loans,Y,Y,0,202500.0,398016.0,22351.5,360000.0,...,1,-4,0.00,765000,0.0,9173.115,183600.000,0.000,0.00,0
970416,328844,0,Cash loans,Y,Y,0,121500.0,808650.0,23643.0,675000.0,...,1,-2,0.00,0,0.0,0.000,0.000,0.000,0.00,0
527629,226156,0,Cash loans,N,Y,0,81000.0,521280.0,26743.5,450000.0,...,1,-4,0.00,225000,0.0,0.000,340.155,0.000,0.00,0
1304271,235159,0,Cash loans,Y,Y,2,247500.0,631332.0,46071.0,585000.0,...,1,-6,0.00,90000,0.0,0.000,0.000,0.000,0.00,0


In [62]:
y_treino_pred_amostra = pipe.predict_proba(df_amostra)[:, 1]

D:\Anaconda\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  warnings.warn("Given feature/column names or counts do not match "


In [63]:
probabilidade = list(y_treino_pred_amostra)

In [64]:
resultado = df_amostra[['SK_ID_CURR']]
resultado['TARGET'] = probabilidade
resultado

<ipython-input-64-951b18530d01>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultado['TARGET'] = probabilidade


,SK_ID_CURR,TARGET
73732,116081,0.040258
890254,313981,0.163453
921161,429701,0.026213
491063,356487,0.047174
970416,328844,0.073787
527629,226156,0.116929
1304271,235159,0.207931


## Dataset teste

In [65]:
df_test = pd.read_csv('application_test_student.csv')

In [66]:
df_test

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,149741,Cash loans,F,N,N,0,117000.0,417024.0,20191.5,360000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
1,363290,Cash loans,M,N,Y,0,450000.0,640080.0,31261.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
2,436006,Revolving loans,M,Y,Y,0,450000.0,900000.0,45000.0,900000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,0.0
3,377703,Cash loans,M,Y,N,1,360000.0,1125000.0,33025.5,1125000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
4,188624,Cash loans,M,Y,Y,0,675000.0,835380.0,42840.0,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61498,102817,Cash loans,M,Y,N,0,225000.0,472500.0,46161.0,454500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
61499,343961,Cash loans,M,N,Y,0,112500.0,508495.5,24592.5,454500.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0
61500,427828,Cash loans,F,Y,Y,0,157500.0,808650.0,23643.0,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
61501,405956,Cash loans,F,N,Y,0,202500.0,590337.0,30271.5,477000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


### Tratando algumas variáveis do modelo

In [67]:
# Pegando os valores da feature CODE GENDER diferentes de XNA
df_test = df_test[df_test['CODE_GENDER'] != 'XNA']

In [68]:
# Removendo outlier da variável AMT_INCOME_TOTAL
df_test = df_test[df_test['AMT_INCOME_TOTAL'] < 20000000]

In [69]:
# Tratando a feature DAYS_EMPLOYED
df_test['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)

In [70]:
# Tratando a feature DAYS_LAST_PHONE_CHANGE
df_test['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

In [71]:
# Categorizando a idade
def categoriza_idade(dia):
        idade_anos = -dia / 365
        if idade_anos < 27: return 1
        elif idade_anos < 40: return 2
        elif idade_anos < 50: return 3
        elif idade_anos < 65: return 4
        elif idade_anos < 99: return 5
        else: return 0
    
df_test['AGE_RANGE'] = df_test['DAYS_BIRTH'].apply(lambda x: categoriza_idade(x))

In [72]:
# Categorizando a feture acompanhantes
def trata_acompanhante_(valor):
    if valor in ['Group of people', 'Other_B']:
        return 1
    elif valor in ['Unaccompanied', 'Other_A']:
        return 2
    elif valor in ['Family', 'Spouse, partner', 'Children']:
        return 3
    else:
        return np.nan
    
df_test['NAME_TYPE_SUITE'] = df_test['NAME_TYPE_SUITE'].apply(lambda x: trata_acompanhante_(x))

In [73]:
# Categorizando a feture tipo de renda
def trata_acompanhante_(valor):
    if valor in ['Working']:
        return 1
    elif valor in ['Commercial associate']:
        return 2
    else:
        return 3
    
df_test['NAME_INCOME_TYPE'] = df_test['NAME_INCOME_TYPE'].apply(lambda x: trata_acompanhante_(x))

In [74]:
def trata_educacao_(valor):
    if valor in ['Lower secondary']:
        return 1
    elif valor in ['Secondary / secondary special', 'Incomplete higher']:
        return 2
    elif valor in ['Higher education']:
        return 3
    elif valor in ['Academic degree']:
        return 4
    else:
        return -1

In [75]:
def trata_educacao(tabela):
    tabela = tabela.copy()
    for i in tabela.columns:
        tabela[i] = tabela[i].apply(trata_educacao_)
    return tabela

In [76]:
df_junto_test = pd.merge(left=df_test, right=df_reduzido, how = 'inner', on = 'SK_ID_CURR')

In [77]:
# Criando uma variavel que representa o quanto o crédito fornecido representa da receita total do devedor
df_junto_test['DEBT_TO_INCOME'] = df_junto_test['AMT_CREDIT']/df_junto_test['AMT_INCOME_TOTAL']

In [78]:
# Criando uma variável que representa o quanto sobra da receita do cliente após pagamento da anuidade
df_junto_test['INTEREST_COVER'] = df_junto_test['AMT_INCOME_TOTAL'] / df_junto_test['AMT_ANNUITY']

In [79]:
# Criando uma variável que representa o quanto do crédito representa preço do ativo
df_junto_test['DOWN_PAYMENT'] = df_junto_test['AMT_CREDIT'] / df_junto_test['AMT_GOODS_PRICE'] -1

In [80]:
# Criando uma variável que representa a porcentagem total da renda por cada membro
df_junto_test['TOTAL_INCOME_BY_MENBERS'] = df_junto_test['AMT_INCOME_TOTAL'] / df_junto_test['CNT_FAM_MEMBERS']

### Dropando algumas features

In [81]:
df_junto_test = df_junto_test.drop(['AGE_RANGE','CODE_GENDER','ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG',
                          'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE',
                          'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI',
                          'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
                          'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13',
                          'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
                          'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
                          'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
                          'AMT_REQ_CREDIT_BUREAU_YEAR'], axis=1)

In [82]:
df_junto_test

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_SUM,CREDIT_TYPE,DAYS_CREDIT_UPDATE,CREDIT_DURATION_DAYS,DEBT_TO_INCOME,INTEREST_COVER,DOWN_PAYMENT,TOTAL_INCOME_BY_MENBERS
0,149741,Cash loans,N,N,0,117000.0,417024.0,20191.5,360000.0,3.0,...,-1648.0,-1863.0,360000.00,Consumer credit,-1493,730.0,3.564308,5.794517,0.158400,58500.0
1,149741,Cash loans,N,N,0,117000.0,417024.0,20191.5,360000.0,3.0,...,1314.0,NaN,0.00,Credit card,-13,1637.0,3.564308,5.794517,0.158400,58500.0
2,149741,Cash loans,N,N,0,117000.0,417024.0,20191.5,360000.0,3.0,...,-448.0,-476.0,135742.50,Consumer credit,-474,579.0,3.564308,5.794517,0.158400,58500.0
3,149741,Cash loans,N,N,0,117000.0,417024.0,20191.5,360000.0,3.0,...,-92.0,-217.0,22891.50,Consumer credit,-207,364.0,3.564308,5.794517,0.158400,58500.0
4,149741,Cash loans,N,N,0,117000.0,417024.0,20191.5,360000.0,3.0,...,-971.0,-1313.0,219118.50,Consumer credit,-588,731.0,3.564308,5.794517,0.158400,58500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291499,343961,Cash loans,N,Y,0,112500.0,508495.5,24592.5,454500.0,3.0,...,-178.0,NaN,391500.00,Credit card,-30,1096.0,4.519960,4.574565,0.118802,56250.0
291500,427828,Cash loans,Y,Y,0,157500.0,808650.0,23643.0,675000.0,2.0,...,558.0,NaN,90000.00,Credit card,-43,1284.0,5.134286,6.661591,0.198000,78750.0
291501,427828,Cash loans,Y,Y,0,157500.0,808650.0,23643.0,675000.0,2.0,...,1415.0,NaN,418950.00,Consumer credit,-43,1826.0,5.134286,6.661591,0.198000,78750.0
291502,427828,Cash loans,Y,Y,0,157500.0,808650.0,23643.0,675000.0,2.0,...,-27.0,-331.0,171000.00,Consumer credit,-43,730.0,5.134286,6.661591,0.198000,78750.0


In [83]:
df_junto_test = pd.merge(left=df_junto_test, right=df_junto_previous, how = 'inner', on = 'SK_ID_CURR')

In [84]:
df_junto_test

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY_x,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_LAST_APPL_PER_CONTRACT,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_TOTAL_RECEIVABLE,SK_DPD_CREDIT
0,363290,Cash loans,N,Y,0,450000.0,640080.0,31261.5,450000.0,2.0,...,1,-1,466429.140,450000,9000.000,23185.620,0.000,449312.085,464953.140,0
1,363290,Cash loans,N,Y,0,450000.0,640080.0,31261.5,450000.0,2.0,...,1,-3,465066.720,450000,13500.000,22874.220,675.045,448034.355,463590.720,0
2,363290,Cash loans,N,Y,0,450000.0,640080.0,31261.5,450000.0,2.0,...,1,-5,340963.470,450000,0.000,18657.450,4443.615,338628.015,340963.470,0
3,363290,Cash loans,N,Y,0,450000.0,640080.0,31261.5,450000.0,2.0,...,1,-4,438096.825,450000,102641.355,18056.385,0.000,435209.400,438096.825,0
4,363290,Cash loans,N,Y,0,450000.0,640080.0,31261.5,450000.0,2.0,...,1,-2,419888.385,450000,9000.000,22397.670,22714.425,416912.085,419888.385,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426897,343961,Cash loans,N,Y,0,112500.0,508495.5,24592.5,454500.0,3.0,...,1,-3,0.000,45000,0.000,0.000,160.650,0.000,0.000,0
426898,343961,Cash loans,N,Y,0,112500.0,508495.5,24592.5,454500.0,3.0,...,1,-6,0.000,45000,0.000,0.000,160.650,0.000,0.000,0
426899,343961,Cash loans,N,Y,0,112500.0,508495.5,24592.5,454500.0,3.0,...,1,-4,0.000,45000,0.000,0.000,160.650,0.000,0.000,0
426900,343961,Cash loans,N,Y,0,112500.0,508495.5,24592.5,454500.0,3.0,...,1,-2,0.000,45000,157.500,0.000,160.650,0.000,0.000,0


In [85]:
df_junto_test.SK_ID_CURR.value_counts()

430261    324
254208    258
323928    252
439457    240
331681    230
         ... 
243426      1
105795      1
285062      1
342698      1
147352      1
Name: SK_ID_CURR, Length: 13406, dtype: int64

In [86]:
df_junto_test_drop = df_junto_test.copy()

In [87]:
# Dropando os valores repetidos
df_junto_test_drop = df_junto_test_drop.drop_duplicates(subset=['SK_ID_CURR', 'MONTHS_BALANCE'], keep='last')

In [88]:
df_junto_test_drop

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY_x,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_LAST_APPL_PER_CONTRACT,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_TOTAL_RECEIVABLE,SK_DPD_CREDIT
30,363290,Cash loans,N,Y,0,450000.0,640080.0,31261.5,450000.0,2.0,...,1,-1,466429.140,450000,9000.000,23185.620,0.000,449312.085,464953.140,0
31,363290,Cash loans,N,Y,0,450000.0,640080.0,31261.5,450000.0,2.0,...,1,-3,465066.720,450000,13500.000,22874.220,675.045,448034.355,463590.720,0
32,363290,Cash loans,N,Y,0,450000.0,640080.0,31261.5,450000.0,2.0,...,1,-5,340963.470,450000,0.000,18657.450,4443.615,338628.015,340963.470,0
33,363290,Cash loans,N,Y,0,450000.0,640080.0,31261.5,450000.0,2.0,...,1,-4,438096.825,450000,102641.355,18056.385,0.000,435209.400,438096.825,0
34,363290,Cash loans,N,Y,0,450000.0,640080.0,31261.5,450000.0,2.0,...,1,-2,419888.385,450000,9000.000,22397.670,22714.425,416912.085,419888.385,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426897,343961,Cash loans,N,Y,0,112500.0,508495.5,24592.5,454500.0,3.0,...,1,-3,0.000,45000,0.000,0.000,160.650,0.000,0.000,0
426898,343961,Cash loans,N,Y,0,112500.0,508495.5,24592.5,454500.0,3.0,...,1,-6,0.000,45000,0.000,0.000,160.650,0.000,0.000,0
426899,343961,Cash loans,N,Y,0,112500.0,508495.5,24592.5,454500.0,3.0,...,1,-4,0.000,45000,0.000,0.000,160.650,0.000,0.000,0
426900,343961,Cash loans,N,Y,0,112500.0,508495.5,24592.5,454500.0,3.0,...,1,-2,0.000,45000,157.500,0.000,160.650,0.000,0.000,0


In [89]:
df_junto_test_drop.SK_ID_CURR.value_counts()

272857    6
238997    6
130334    6
146726    6
177463    6
         ..
345083    1
258228    1
166911    1
195976    1
137547    1
Name: SK_ID_CURR, Length: 13406, dtype: int64

In [90]:
df_treino = df_junto_drop
df_teste = df_junto_test_drop

x_treino = df_treino.loc[:, var_edu + var_cat + var_num]
x_teste = df_teste.loc[:, var_edu + var_cat + var_num]

y_treino = df_treino.loc[:, 'TARGET']


In [91]:
var_cat = ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE', 'NAME_FAMILY_STATUS', 
           'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
           'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE', 'CREDIT_TYPE', 'NAME_CONTRACT_STATUS',
           'CODE_REJECT_REASON', 'NAME_PORTFOLIO', 'NAME_YIELD_GROUP']
var_edu = ['NAME_EDUCATION_TYPE']
var_num = [i for i in df_junto_test_drop.columns[2:] if df_junto_test_drop[i].dtype != 'object']

In [92]:
pipe_edu = Pipeline(steps = [
    ('educacao', FunctionTransformer(trata_educacao))
])

# para não ser necessário fazer o impute dos missing, transformo em string e trato como uma nova categoria
pipe_cat = Pipeline(steps = [
    ('para_string', FunctionTransformer(lambda x: x.astype(str))), 
    ('dummies', OneHotEncoder(drop = 'if_binary', sparse = False)),
])

pipe_num = Pipeline(steps = [
    ('redimensionamento', StandardScaler()),
    ('imputer', SimpleImputer(strategy = 'median', add_indicator = 'True'))
])

trata_variaveis = ColumnTransformer(transformers = [
    ('pipe_edu', pipe_edu, var_edu),
    ('pipe_cat', pipe_cat, var_cat),
    ('pipe_num', pipe_num, var_num)
])

pipe = Pipeline(steps = [
    ('pre_processamento', trata_variaveis),
    ('feature_selection', SelectFromModel(LogisticRegression(penalty = 'l1', 
                                                             solver = 'liblinear', 
                                                             C = 0.01))),
    
    ('modelo', RandomForestClassifier(
        criterion='entropy',
        n_estimators = 100,
        max_samples = 0.9,
        max_features = 0.6,
        max_depth = 9,
        n_jobs = 10,
        random_state = 1))
])

In [93]:
%%time
pipe.fit(x_treino, y_treino)

Wall time: 1min 43s


Pipeline(steps=[('pre_processamento',
                 ColumnTransformer(transformers=[('pipe_edu',
                                                  Pipeline(steps=[('educacao',
                                                                   FunctionTransformer(func=<function trata_educacao at 0x000001A90D9D8430>))]),
                                                  ['NAME_EDUCATION_TYPE']),
                                                 ('pipe_cat',
                                                  Pipeline(steps=[('para_string',
                                                                   FunctionTransformer(func=<function <lambda> at 0x000001A90CF68820>)),
                                                                  ('dummies',
                                                                   OneHotEncoder(drop='if...
                                                   'LIVE_REGION_NOT_WORK_REGION',
                                                   'REG_CITY_NOT_L

In [94]:
y_treino_pred = pipe.predict_proba(x_treino)[:, 1]

In [95]:
roc_auc_score(y_treino, y_treino_pred)

0.8724888142415166

In [105]:
df_junto_test_drop.SK_ID_CURR.value_counts()

272857    6
238997    6
130334    6
146726    6
177463    6
         ..
345083    1
258228    1
166911    1
195976    1
137547    1
Name: SK_ID_CURR, Length: 13406, dtype: int64

In [97]:
y_teste_pred = pipe.predict_proba(x_teste)[:, 1]

In [98]:
y_teste_pred.shape

(71781,)

In [99]:
probabilidade = list(y_teste_pred)

In [103]:
resultado_test = df_junto_test_drop[['SK_ID_CURR']]
resultado_test['TARGET'] = probabilidade
resultado_test
# Salvando o dataset com o resultado do nosso modelo
resultado_test.to_csv('resultado_test.csv', index = False)

<ipython-input-103-3aabe65584d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultado_test['TARGET'] = probabilidade
